In [1]:
import os
import requests
from datetime import datetime

import geopandas as gpd
import geojsonio

import pandas as pd
from pandas.io.json import json_normalize

import folium
from folium import plugins

## Schema of USGS data

## Getting data

In [2]:
r = requests.get("https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_day.geojson")
data = r.json()

In [3]:
date_columns = ['Year', 'Mo', 'Dy', 'Hr', 'Mn', 'Sec']
df = pd.read_csv('data/tsunamis-2020-11-28_15-17-05_-0500.tsv', sep = '\t')
# df['dtg'] = df.to_datetime(date_columns)

# df

In [4]:
dates = pd.DataFrame({'year': df.Year, 'month': df.Mo, 'day': df.Dy, })

# ['Year', 'Mo', 'Dy', 'Hr', 'Mn', 'Sec']
df

,Search Parameters,Year,Mo,Dy,Hr,Mn,Sec,Tsunami Event Validity,Tsunami Cause Code,Earthquake Magnitude,...,Total Missing,Total Missing Description,Total Injuries,Total Injuries Description,Total Damage ($Mil),Total Damage Description,Total Houses Destroyed,Total Houses Destroyed Description,Total Houses Damaged,Total Houses Damaged Description
0,"[""1993 <= Year >= 2020""]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1993.0,2.0,7.0,13.0,27.0,42.0,4.0,1.0,6.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1993.0,3.0,12.0,14.0,1.0,35.4,3.0,1.0,6.4,...,NaN,NaN,20.0,1.0,NaN,1.0,NaN,2.0,NaN,2.0
3,NaN,1993.0,6.0,8.0,13.0,3.0,36.4,4.0,1.0,7.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1993.0,6.0,18.0,11.0,52.0,54.8,1.0,1.0,6.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,NaN,2020.0,6.0,18.0,12.0,49.0,53.0,4.0,1.0,7.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
352,NaN,2020.0,6.0,23.0,15.0,29.0,5.0,4.0,1.0,7.4,...,NaN,NaN,NaN,1.0,NaN,4.0,NaN,NaN,8123.0,4.0
353,NaN,2020.0,7.0,22.0,6.0,12.0,44.0,4.0,1.0,7.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
354,NaN,2020.0,10.0,19.0,20.0,54.0,39.0,4.0,1.0,7.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.drop(index=df.iloc[:1, :].index.tolist(), inplace=True)
df.drop(['Search Parameters'], axis=1, inplace=True)

In [6]:
df

,Year,Mo,Dy,Hr,Mn,Sec,Tsunami Event Validity,Tsunami Cause Code,Earthquake Magnitude,Vol,...,Total Missing,Total Missing Description,Total Injuries,Total Injuries Description,Total Damage ($Mil),Total Damage Description,Total Houses Destroyed,Total Houses Destroyed Description,Total Houses Damaged,Total Houses Damaged Description
1,1993.0,2.0,7.0,13.0,27.0,42.0,4.0,1.0,6.3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1993.0,3.0,12.0,14.0,1.0,35.4,3.0,1.0,6.4,NaN,...,NaN,NaN,20.0,1.0,NaN,1.0,NaN,2.0,NaN,2.0
3,1993.0,6.0,8.0,13.0,3.0,36.4,4.0,1.0,7.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1993.0,6.0,18.0,11.0,52.0,54.8,1.0,1.0,6.6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1993.0,7.0,12.0,13.0,17.0,11.9,4.0,1.0,7.7,NaN,...,NaN,NaN,233.0,3.0,1207.0,4.0,2374.0,4.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,2020.0,6.0,18.0,12.0,49.0,53.0,4.0,1.0,7.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
352,2020.0,6.0,23.0,15.0,29.0,5.0,4.0,1.0,7.4,NaN,...,NaN,NaN,NaN,1.0,NaN,4.0,NaN,NaN,8123.0,4.0
353,2020.0,7.0,22.0,6.0,12.0,44.0,4.0,1.0,7.8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
354,2020.0,10.0,19.0,20.0,54.0,39.0,4.0,1.0,7.6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df['dtg'] = df[['Year', 'Mo', 'Dy', 'Hr', 'Mn', 'Sec']].apply(pd.to_datetime, format='%Y-%m-%d %H:%M:%S')



ValueError: Wrong number of items passed 6, placement implies 1

In [ ]:
df = pd.read_csv('data/tsunamis-2020-11-28_15-17-05_-0500.tsv', sep = '\t', parse_dates={'dtg' :['Year', 'Mo', 'Dy', 'Hr', 'Mn', 'Sec']})
df.drop(index=df.iloc[:1, :].index.tolist(), inplace=True)
df.drop(['Search Parameters'], axis=1, inplace=True)

In [ ]:
df.head(3)

In [ ]:
df['dtg'].to_datetime(df['dtg'])

## Parsing the data

In [ ]:
quakes = pd.json_normalize(data=data['features'], meta=['mag','place','time','updated','tz','url','detail','felt','cdi','mmi','alert','status','tsunami','sig','net','code','ids','sources','types','nst','dmin','rms','gap','magType'])

There are a lot of columns that are essientially useless

In [ ]:
quakes.columns = quakes.columns.map(lambda x: x.split(".")[-1])

quakes.drop(['type', 'updated', 'tz', 'url', 'detail', 'felt', 'cdi', 'mmi', 'alert', 'status', 'net', 
                     'code', 'sources', 'types', 'nst', 'dmin', 'rms', 'gap', 'magType' ], axis=1, inplace=True)

Breakout the coords column

In [ ]:
quakes['longitude'] = quakes.coordinates.str[0]
quakes['latitude'] = quakes.coordinates.str[1]
quakes['depth'] = quakes.coordinates.str[2]

quakes.drop(['coordinates', 'title'], axis=1, inplace=True)

In [ ]:
quakes['datetime'] = pd.to_datetime(quakes['time']).dt.strftime("%Y-%m-%d %H:%M")
quakes.drop(['time'], axis=1, inplace=True)

In [ ]:
quakes.round({'longitude': 5, 'latitude': 5, 'depth': 0})
quakes.head(2)

In [ ]:
m = folium.Map(
    location=[0, 0],
    tiles='cartodbpositron',
    zoom_start=1.8
)

Figure out how to get time to display

In [ ]:
fmtr = "function(num) {return L.Util.formatNum(num, 3);};"
folium.plugins.MousePosition(separator=' / ', prefix="Lat/Long: ", lat_formatter=fmtr, lng_formatter=fmtr).add_to(m)

In [ ]:
for i, row in quakes.iterrows():
    folium.CircleMarker((row.latitude, row.longitude), radius=row.mag*2.2, 
                        color='red', weight = 0, opacity =.4, fill=True, fill_color='orange', fill_opacity=.3, 
                        popup=[f"Time: {row.datetime},\n Mag: {row.mag},\n Depth: {row.depth} km"]).add_to(m)
    

In [ ]:
boundaries = ('data/PB2002_boundaries.json')
line_style = {'color': '#FF3333', 'weight': 1, 'opacity':row.mag*1.4}


folium.GeoJson(
    boundaries,
    name='major fault lines',
    style_function=lambda x:line_style,
    smooth_factor=4.0,
      
).add_to(m)
m